In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from unityagents import UnityEnvironment
import random, pdb
import torch
import numpy as np
from collections import deque
#from prioritised_dqn_agent import Agent
from dqn_agent import Agent

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()

# Setup

In [ ]:
# Create environment
env = UnityEnvironment(file_name="Banana.app")

In [ ]:
# Get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# Grab environment details
action_size = brain.vector_action_space_size
example_state = env_info.vector_observations[0]
state_size = len(example_state)

# Create the agent
agent = Agent(state_size=state_size, action_size=action_size, seed=0)

# Train

In [ ]:
def dqn(n_episodes=2000, max_t=10000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    average_score = -np.inf
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        
        # Perform an episode
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            
            # Direct the agent to choose an action
            action = agent.act(state, eps)
            
            # Interact with the environment
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            
            # Feed the outcome back to the agent
            agent.step(state, action, reward, next_state, done)
            
            # Advance
            state = next_state
            score += reward
            if done:
                break
                
        # Update the score
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        average_score = np.mean(scores_window)
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
        # Output
        if i_episode % 1 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, average_score), end="")
        if i_episode % 50 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, average_score))
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, average_score))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

scores = dqn(n_episodes=500, eps_start=1.0, eps_end=0.01)

# Plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
scores_1 = scores